In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
%matplotlib inline

In [43]:
complaint_df = pd.read_csv('data/complaints-cpd-2016-oct.csv', dtype={'ACTION_CODE': str})
print(complaint_df.info(verbose=True))
complaint_df.head()

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (0,7) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 232039 entries, 0 to 232038
Data columns (total 16 columns):
COMPLAINT_NUMBER                   232039 non-null object
INCIDENT_DATE                      231977 non-null object
COMPLAINT_DATE                     232039 non-null object
CLOSED_DATE                        231325 non-null object
ACCUSEDOFFICER_FNAME               232039 non-null object
ACCUSEDOFFICER_LNAME               232039 non-null object
ACCUSEDOFFICER_FULL_NAME           232039 non-null object
ACCUSEDOFFICER_UNIT                124803 non-null object
ACCUSEDOFFICER_APPOINTMENT_DATE    231854 non-null object
INITIAL_CATEGORY_CODE              232039 non-null object
INITIAL_CATEGORY                   232039 non-null object
CURRENT_CATEGORY_CODE              232020 non-null object
CURRENT_CATEGORY                   232039 non-null object
ACTION_CODE                        228402 non-null object
ACTION_DESCRIPTION                 228402 non-null object
FINDING_CODE       

,COMPLAINT_NUMBER,INCIDENT_DATE,COMPLAINT_DATE,CLOSED_DATE,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,INITIAL_CATEGORY_CODE,INITIAL_CATEGORY,CURRENT_CATEGORY_CODE,CURRENT_CATEGORY,ACTION_CODE,ACTION_DESCRIPTION,FINDING_CODE
0,237004,21-MAY-1997,21-MAY-1997,25-MAY-1999,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,05-DEC-1994,05A,ARRESTEE - DURING ARREST,05B,ARRESTEE - DURING ARREST,600,NO ACTION TAKEN,UN
1,258124,19-NOV-1999,20-NOV-1999,07-DEC-2000,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,05-DEC-1994,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,04E,PRISONERS PROPERTY - INVENTORY/RECEIPT,600,NO ACTION TAKEN,NS
2,190369,11-FEB-1992,11-FEB-1992,09-APR-1992,CARMEL,ABBATE,"ABBATE, CARMEL G",641,06-JAN-1969,03G,MISCELLANEOUS,03G,MISCELLANEOUS,600,NO ACTION TAKEN,UN
3,210426,30-JUL-1994,14-JUL-1994,01-DEC-1994,CARMEL,ABBATE,"ABBATE, CARMEL G",640,06-JAN-1969,10V,INVENTORY PROCEDURES,04E,INVENTORY PROCEDURES,600,NO ACTION TAKEN,NS
4,219424,12-JUL-1995,19-JUL-1995,23-JAN-1996,CARMEL,ABBATE,"ABBATE, CARMEL G",640,06-JAN-1969,09J,MISCELLANEOUS,05A,MISCELLANEOUS,600,NO ACTION TAKEN,NS


In [10]:
# Reading in CPD context data on employees from chicago-police-data repo
cpd_employees = pd.read_csv('../chicago-police-data/context_data/CPD Employees, one row per individual.csv',
                           dtype={'DOBYEAR': str, 'CURRAGE': str, 'CPD_UNIT_ASSIGNED_NO': str})
cpd_employees['APPOINTED_DATE'] = cpd_employees['APPOINTED_DATE'].str.upper()
cpd_employees['RESIGNATION_DATE'] = cpd_employees['RESIGNATION_DATE'].str.upper()
print(cpd_employees.info(verbose=True))
cpd_employees[cpd_employees.columns.values.tolist()[:-7]].head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31710 entries, 0 to 31709
Data columns (total 24 columns):
LAST_NME                31710 non-null object
FIRST_NME               31710 non-null object
MIDDLE_INITIAL          26091 non-null object
SEX_CODE_CD             31710 non-null object
RACE                    31710 non-null object
DOBYEAR                 31708 non-null object
CURRAGE                 31708 non-null object
STATUS_I                31710 non-null object
APPOINTED_DATE          31704 non-null object
EMPLOYEE_POSITION_CD    31710 non-null object
DESCR                   31710 non-null object
CPD_UNIT_ASSIGNED_NO    30869 non-null object
UNITDESCR               30869 non-null object
RESIGNATION_DATE        19304 non-null object
STAR1                   27234 non-null float64
STAR2                   9189 non-null float64
STAR3                   3088 non-null float64
STAR4                   928 non-null float64
STAR5                   225 non-null float64
STAR6             

,LAST_NME,FIRST_NME,MIDDLE_INITIAL,SEX_CODE_CD,RACE,DOBYEAR,CURRAGE,STATUS_I,APPOINTED_DATE,EMPLOYEE_POSITION_CD,DESCR,CPD_UNIT_ASSIGNED_NO,UNITDESCR,RESIGNATION_DATE,STAR1,STAR2,STAR3
0,AARON,JEFFERY,M,M,WHITE,1971,45,Y,26-SEP-05,9161,POLICE OFFICER,014,DISTRICT 014,NaN,13001.0,NaN,NaN
1,AARON,KARINA,NaN,F,WHITE HISPANIC,1980,35,Y,26-SEP-05,9161,POLICE OFFICER,015,DISTRICT 015,NaN,17545.0,13705.0,NaN
2,ABATE,DANIEL,P,M,WHITE,1942,74,N,15-JUN-70,9161,POLICE OFFICER,543,DETACHED SERVICES - MISCELLANEOUS DETAIL,15-FEB-04,14535.0,3023.0,2628.0
3,ABBATE,ANTHONY,G,M,WHITE,1968,47,N,5-DEC-94,9161,POLICE OFFICER,020,DISTRICT 020,22-DEC-09,18601.0,NaN,NaN
4,ABBATE,CARMEL,G,M,WHITE,1942,73,N,6-JAN-69,9165,PO AS DETECTIVE,640,DETECTIVE SECTION - AREA 4,15-AUG-03,20484.0,NaN,NaN


In [17]:
cpd_employees['APPOINTED_DATE'] = cpd_employees['APPOINTED_DATE'].fillna('')
cpd_employees['RESIGNATION_DATE'] = cpd_employees['RESIGNATION_DATE'].fillna('')
cpd_employees.head()

,LAST_NME,FIRST_NME,MIDDLE_INITIAL,SEX_CODE_CD,RACE,DOBYEAR,CURRAGE,STATUS_I,APPOINTED_DATE,EMPLOYEE_POSITION_CD,...,STAR1,STAR2,STAR3,STAR4,STAR5,STAR6,STAR7,STAR8,STAR9,STAR10
0,AARON,JEFFERY,M,M,WHITE,1971,45,Y,26-SEP-05,9161,...,13001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AARON,KARINA,NaN,F,WHITE HISPANIC,1980,35,Y,26-SEP-05,9161,...,17545.0,13705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABATE,DANIEL,P,M,WHITE,1942,74,N,15-JUN-70,9161,...,14535.0,3023.0,2628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBATE,ANTHONY,G,M,WHITE,1968,47,N,5-DEC-94,9161,...,18601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBATE,CARMEL,G,M,WHITE,1942,73,N,6-JAN-69,9165,...,20484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
cpd_employees['APPOINTED_DATE'] = cpd_employees['APPOINTED_DATE'].apply(lambda x: '0' + x if len(str(x)) < 9 and x is not '' else x)
cpd_employees['RESIGNATION_DATE'] = cpd_employees['RESIGNATION_DATE'].apply(lambda x: '0' + x if len(str(x)) < 9 and x is not '' else x)
cpd_employees.head()

,LAST_NME,FIRST_NME,MIDDLE_INITIAL,SEX_CODE_CD,RACE,DOBYEAR,CURRAGE,STATUS_I,APPOINTED_DATE,EMPLOYEE_POSITION_CD,...,STAR1,STAR2,STAR3,STAR4,STAR5,STAR6,STAR7,STAR8,STAR9,STAR10
0,AARON,JEFFERY,M,M,WHITE,1971,45,Y,26-SEP-05,9161,...,13001.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,AARON,KARINA,NaN,F,WHITE HISPANIC,1980,35,Y,26-SEP-05,9161,...,17545.0,13705.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,ABATE,DANIEL,P,M,WHITE,1942,74,N,15-JUN-70,9161,...,14535.0,3023.0,2628.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ABBATE,ANTHONY,G,M,WHITE,1968,47,N,05-DEC-94,9161,...,18601.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ABBATE,CARMEL,G,M,WHITE,1942,73,N,06-JAN-69,9165,...,20484.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [19]:
cpd_employees = cpd_employees.fillna('')
cpd_employees.head()

,LAST_NME,FIRST_NME,MIDDLE_INITIAL,SEX_CODE_CD,RACE,DOBYEAR,CURRAGE,STATUS_I,APPOINTED_DATE,EMPLOYEE_POSITION_CD,...,STAR1,STAR2,STAR3,STAR4,STAR5,STAR6,STAR7,STAR8,STAR9,STAR10
0,AARON,JEFFERY,M,M,WHITE,1971,45,Y,26-SEP-05,9161,...,13001,,,,,,,,,
1,AARON,KARINA,,F,WHITE HISPANIC,1980,35,Y,26-SEP-05,9161,...,17545,13705,,,,,,,,
2,ABATE,DANIEL,P,M,WHITE,1942,74,N,15-JUN-70,9161,...,14535,3023,2628,,,,,,,
3,ABBATE,ANTHONY,G,M,WHITE,1968,47,N,05-DEC-94,9161,...,18601,,,,,,,,,
4,ABBATE,CARMEL,G,M,WHITE,1942,73,N,06-JAN-69,9165,...,20484,,,,,,,,,


In [20]:
#cpd_employees.to_csv('data/cpd-employees-individuals.csv',index=False)

In [50]:
cpd_employees_merge = cpd_employees[['FIRST_NME', 'LAST_NME', 'SEX_CODE_CD', 'RACE', 'DOBYEAR', 'STATUS_I', 'APPOINTED_DATE',
                                     'EMPLOYEE_POSITION_CD', 'DESCR', 'CPD_UNIT_ASSIGNED_NO', 'UNITDESCR', 'RESIGNATION_DATE',
                                     'STAR1', 'STAR2', 'STAR3', 'STAR4', 'STAR5', 'STAR6', 'STAR7', 'STAR8', 'STAR9', 'STAR10']].copy()
cpd_employees_merge.head()

,FIRST_NME,LAST_NME,SEX_CODE_CD,RACE,DOBYEAR,STATUS_I,APPOINTED_DATE,EMPLOYEE_POSITION_CD,DESCR,CPD_UNIT_ASSIGNED_NO,...,STAR1,STAR2,STAR3,STAR4,STAR5,STAR6,STAR7,STAR8,STAR9,STAR10
0,JEFFERY,AARON,M,WHITE,1971,Y,26-SEP-05,9161,POLICE OFFICER,014,...,13001,,,,,,,,,
1,KARINA,AARON,F,WHITE HISPANIC,1980,Y,26-SEP-05,9161,POLICE OFFICER,015,...,17545,13705,,,,,,,,
2,DANIEL,ABATE,M,WHITE,1942,N,15-JUN-70,9161,POLICE OFFICER,543,...,14535,3023,2628,,,,,,,
3,ANTHONY,ABBATE,M,WHITE,1968,N,05-DEC-94,9161,POLICE OFFICER,020,...,18601,,,,,,,,,
4,CARMEL,ABBATE,M,WHITE,1942,N,06-JAN-69,9165,PO AS DETECTIVE,640,...,20484,,,,,,,,,


In [55]:
complaint_officers = complaint_df.drop_duplicates(subset=['ACCUSEDOFFICER_FNAME', 'ACCUSEDOFFICER_LNAME',
                                                          'ACCUSEDOFFICER_APPOINTMENT_DATE']).copy()
print(complaint_df.shape)
print(complaint_officers.shape)
complaint_officers = complaint_officers[['ACCUSEDOFFICER_FNAME', 'ACCUSEDOFFICER_LNAME', 'ACCUSEDOFFICER_FULL_NAME',
                                         'ACCUSEDOFFICER_UNIT', 'ACCUSEDOFFICER_APPOINTMENT_DATE']].copy()
# Change datetime format for merge on appointment date
complaint_officers['ORIGIN_APPOINTMENT_DATE'] = complaint_officers['ACCUSEDOFFICER_APPOINTMENT_DATE']
complaint_officers['ACCUSEDOFFICER_APPOINTMENT_DATE'] = complaint_officers['ACCUSEDOFFICER_APPOINTMENT_DATE'].fillna('')
complaint_officers['ACCUSEDOFFICER_APPOINTMENT_DATE'] = complaint_officers['ACCUSEDOFFICER_APPOINTMENT_DATE'].apply(lambda x: re.sub(r'(-19)|(-20)', '-', x))
complaint_officers.head()

(232039, 16)
(24733, 16)


,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,ORIGIN_APPOINTMENT_DATE
0,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,05-DEC-94,05-DEC-1994
2,CARMEL,ABBATE,"ABBATE, CARMEL G",641,06-JAN-69,06-JAN-1969
8,CARMEN,ABBATE,"ABBATE, CARMEN S",25,16-OCT-54,16-OCT-1954
12,TERRY,ABBATE,"ABBATE, TERRY M",12,04-DEC-95,04-DEC-1995
15,MICHAEL,ABBEY,"ABBEY, MICHAEL J",151,26-APR-76,26-APR-1976


In [56]:
complaint_cpd_employee_merge = complaint_officers.merge(cpd_employees_merge, 
                                                        left_on=['ACCUSEDOFFICER_FNAME', 'ACCUSEDOFFICER_LNAME',
                                                                 'ACCUSEDOFFICER_APPOINTMENT_DATE'],
                                                        right_on=['FIRST_NME', 'LAST_NME', 'APPOINTED_DATE'],
                                                        how='left')
print(complaint_cpd_employee_merge.info(verbose=True))
complaint_cpd_employee_merge.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24758 entries, 0 to 24757
Data columns (total 28 columns):
ACCUSEDOFFICER_FNAME               24758 non-null object
ACCUSEDOFFICER_LNAME               24758 non-null object
ACCUSEDOFFICER_FULL_NAME           24758 non-null object
ACCUSEDOFFICER_UNIT                19119 non-null object
ACCUSEDOFFICER_APPOINTMENT_DATE    24758 non-null object
ORIGIN_APPOINTMENT_DATE            24673 non-null object
FIRST_NME                          21934 non-null object
LAST_NME                           21934 non-null object
SEX_CODE_CD                        21934 non-null object
RACE                               21934 non-null object
DOBYEAR                            21934 non-null object
STATUS_I                           21934 non-null object
APPOINTED_DATE                     21934 non-null object
EMPLOYEE_POSITION_CD               21934 non-null object
DESCR                              21934 non-null object
CPD_UNIT_ASSIGNED_NO               2

,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,ORIGIN_APPOINTMENT_DATE,FIRST_NME,LAST_NME,SEX_CODE_CD,RACE,...,STAR1,STAR2,STAR3,STAR4,STAR5,STAR6,STAR7,STAR8,STAR9,STAR10
0,ANTHONY,ABBATE,"ABBATE, ANTHONY G",11,05-DEC-94,05-DEC-1994,ANTHONY,ABBATE,M,WHITE,...,18601,,,,,,,,,
1,CARMEL,ABBATE,"ABBATE, CARMEL G",641,06-JAN-69,06-JAN-1969,CARMEL,ABBATE,M,WHITE,...,20484,,,,,,,,,
2,CARMEN,ABBATE,"ABBATE, CARMEN S",25,16-OCT-54,16-OCT-1954,CARMEN,ABBATE,M,WHITE,...,,,,,,,,,,
3,TERRY,ABBATE,"ABBATE, TERRY M",12,04-DEC-95,04-DEC-1995,TERRY,ABBATE,M,WHITE,...,19596,,,,,,,,,
4,MICHAEL,ABBEY,"ABBEY, MICHAEL J",151,26-APR-76,26-APR-1976,MICHAEL,ABBEY,M,BLACK,...,16511,,,,,,,,,


In [57]:
complaint_cpd_employee_merge.loc[complaint_cpd_employee_merge['FIRST_NME'].isnull()]

,ACCUSEDOFFICER_FNAME,ACCUSEDOFFICER_LNAME,ACCUSEDOFFICER_FULL_NAME,ACCUSEDOFFICER_UNIT,ACCUSEDOFFICER_APPOINTMENT_DATE,ORIGIN_APPOINTMENT_DATE,FIRST_NME,LAST_NME,SEX_CODE_CD,RACE,...,STAR1,STAR2,STAR3,STAR4,STAR5,STAR6,STAR7,STAR8,STAR9,STAR10
10,RASHAD,ABDULLAH,"ABDULLAH, RASHAD",173,13-MAY-91,13-MAY-1991,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
12,KENNETH,ABELS,"ABELS, KENNETH L",8,16-OCT-12,16-OCT-2012,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,ROBERT,ABERNATHY,"ABERNATHY, ROBERT",163,06-DEC-99,06-DEC-1999,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
18,JOHN,ABRAMS,"ABRAMS, JOHN",176,24-MAY-82,24-MAY-1982,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
29,MANUEL,ACEVEDO,"ACEVEDO, MANUEL F",740,01-JAN-03,01-JAN-2003,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
37,PATRICIA,ACKER,"ACKER, PATRICIA T",8,16-MAR-92,16-MAR-1992,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
40,JOSE,ACOSTA,"ACOSTA, JOSE F",25,11-DEC-95,11-DEC-1995,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
42,LISA,ACOSTA,"ACOSTA, LISA A",86,03-OCT-94,03-OCT-1994,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
59,CHARLES,ADAMS,"ADAMS, CHARLES",18,01-OCT-80,01-OCT-1980,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
73,MAURICE,ADAMS,"ADAMS, MAURICE R",168,01-MAY-95,01-MAY-1995,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [58]:
complaint_cpd_employee_merge.to_csv('data/complaints-cpd-2016-oct-context-data.csv', index=False)